# **Wikipedia Scrapper**

Permet d'ajouter une description aux éléments d'un fichier depuis la page wikipedia dediée. <br>

Le fichier doit etre formaté comme suit :
  - Pas d'en-tete ni d'index
  - Première colonne : identifiant ou laisser vide
  - Deuxième colonne : nom ou laisser vide
  - Troisième colonne : nom de l'élément recherché separé par un underscrore (par exemple : michael_jackson)
  - Quatrième colonne : Laisser vide. C'est ici que seront stockées les nouvelles descriptions <br>

Dans le code ci-dessous, veuillez changer les variables ci-dessous afin qu'elles correspondent à vos fichiers.

  - ***nom_fichier_csv*** : nom du fichier d'entrée
  - ***dossier*** : chemin du dossier dans le lequel se trouve le fichier d'entrée
  - ***nom_nouveau_fichier*** : nom du fichier de sortie 
  - ***defaut*** : il se peut qu'il n'existe pas de page Wikipedia pour certains éléments de votre fichier. La variable defaut sert donc de texte par defaut dans ces cas là. <br>


In [1]:
nom_nouveau_fichier = "chanteurs_description.csv"
dossier = "/content/"
nom_fichier_csv ="chanteurs.csv"
defaut = "vide"

Par defaut, si une page Wikipedia n'existe pas en français pour élément donné, c'est la page en anglais (si elle existe) qui sera ajoutée comme description. <br>

Si cela ne vous correspond pas, changez la variable ci-dessous par "False".

In [ ]:
description_en_anglais = True

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import json
import urllib.request

# Liens nécessaires à l'utilisation de l'API de Wikipedia

url_fr = "https://fr.wikipedia.org/w/api.php?action=opensearch&format=json&search="
url_an = "https://en.wikipedia.org/w/api.php?action=opensearch&format=json&search="
url_result_an = "https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exintro&explaintext&redirects=1&format=json&titles="
url_result_fr = "https://fr.wikipedia.org/w/api.php?action=query&prop=extracts&exintro&explaintext&redirects=1&format=json&titles="

In [ ]:
# Lecture du fichier csv et transformation en tableau numpy
file = pd.read_csv(dossier+nom_fichier_csv, header=None, names=["id", "nom", "nom_fichier", "description"])
array = np.asarray(file)


# Permet de récuperer le fichier json correspondant à la page Wikipedia de nom_fichier
def getJson(nom_fichier, url_langue):
    URL = url_langue + nom_fichier
    with urllib.request.urlopen(URL) as url:
        data = json.loads(url.read().decode())
    return data


for line in array:
    if (getJson(line[2].capitalize(), url_fr)[3] == []) and description_en_anglais : # si la page francaise pour cette ligne est vide et si la description_en_anglais est activée
        line[3] = getJson(line[2].capitalize(), url_an)
    else :
        line[3] = getJson(line[2].capitalize(), url_fr)


def getInfo(liste_json):
    name = liste_json[0]
    if "https://en." in liste_json[3][0]:
        data = getJson(name, url_result_an)
    else:
        data = getJson(name, url_result_fr)
    return data


# Affiche les fichiers json de manière formatée
def printJson(dat):
    print(json.dumps(dat, indent=4)) 


# Récupere le contenu depuis wikipedia
def content (list_json):
    key = list(list_json["query"]["pages"].keys())[0]
    try:
        text = list_json["query"]["pages"][key]["extract"]
        tokens = text.replace("\n", ".").replace("\t","")
        return tokens
    except KeyError:
        return ""


# Remplace les colonnes description du fichier d'entrée par les contenus Wikipedia
# Si aucune page n'a été trouvée pour un élément, la case sera remplie par defaut
for line in array:
    if line[3][3] == []:
        line[3] = "vide"
    else:
        lis_json = getInfo(line[3])
        line[3] = content(lis_json)  


# Enregistre le nouveau fichier dans le même repertoire que le fichier de base
pd.DataFrame(array).to_csv(dossier+nom_nouveau_fichier, header=None)